# $\text{Importando módulos e bibliotecas / Coleta de Arquivos}$

In [1]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import numpy as np
from biosppy.signals import ecg


In [2]:
## 1: pegar os caminhos dos arquivos
aurorabp_o_data = 'aurorabp-sample-data-main/sample/measurements_oscillometric'
patient_meas = [f'o00{i}' for i in range(6)]  # o000 até o005

# Lista para armazenar os caminhos completos dos arquivos
tsv_paths = []

for patient in patient_meas:
    path = os.path.join(aurorabp_o_data, patient)
    patient_files = glob.glob(os.path.join(path, '*.tsv'))
    tsv_paths.extend(patient_files)  # adiciona à lista

# Visualizar os caminhos encontrados
for file_path in tsv_paths:
    print(file_path)


aurorabp-sample-data-main/sample/measurements_oscillometric\o000\o000.ambulatory.measurement_23.tsv
aurorabp-sample-data-main/sample/measurements_oscillometric\o000\o000.ambulatory.measurement_25.tsv
aurorabp-sample-data-main/sample/measurements_oscillometric\o000\o000.ambulatory.measurement_27.tsv
aurorabp-sample-data-main/sample/measurements_oscillometric\o000\o000.ambulatory.measurement_28.tsv
aurorabp-sample-data-main/sample/measurements_oscillometric\o000\o000.ambulatory.measurement_30.tsv
aurorabp-sample-data-main/sample/measurements_oscillometric\o000\o000.ambulatory.measurement_33.tsv
aurorabp-sample-data-main/sample/measurements_oscillometric\o000\o000.ambulatory.measurement_35.tsv
aurorabp-sample-data-main/sample/measurements_oscillometric\o000\o000.ambulatory.measurement_37.tsv
aurorabp-sample-data-main/sample/measurements_oscillometric\o000\o000.ambulatory.measurement_38.tsv
aurorabp-sample-data-main/sample/measurements_oscillometric\o000\o000.ambulatory.measurement_39.tsv


In [3]:
#salva a lista numa variavel
filepath = tsv_paths

print(len(filepath)) #retorna a quantidade de itens presentes na lista

306


# $\text{Leitura do arquivo de Features}$

In [4]:
#carregando o df de features do ABP (features.tsv)
features = 'aurorabp-sample-data-main/sample/features.tsv'
#data = os.path.join(features)

df = pd.read_csv(features, sep='\t')

df

,pid,phase,measurement,date_time,sbp,dbp,baseline_sbp,baseline_dbp,delta_sbp,delta_dbp,...,delta_dicrotic_notch_height_pressure,delta_dpdt_pressure,delta_ejection_duration_fraction_pressure,delta_ejection_duration_pressure,delta_hr_pressure,delta_quality_pressure,delta_rwat_pressure,delta_sysrise_pressure,delta_invpat_pressure,delta_rpat_pressure
0,o000,ambulatory,measurement 23,2018-01-01 12:17:45,NaN,NaN,135.5,93.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.753411,NaN,NaN,NaN,NaN
1,o000,ambulatory,measurement 25,2018-01-01 12:47:27,141.0,88.0,135.5,93.0,5.5,-5.0,...,-0.142595,-0.646054,0.034060,-0.0325,18.043242,0.166928,-0.035,-0.0350,-0.339737,0.00925
2,o000,ambulatory,measurement 27,2018-01-01 13:17:30,139.0,93.0,135.5,93.0,3.5,0.0,...,NaN,NaN,NaN,NaN,NaN,-0.753411,NaN,NaN,NaN,NaN
3,o000,ambulatory,measurement 28,2018-01-01 13:46:55,NaN,NaN,135.5,93.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.753411,NaN,NaN,NaN,NaN
4,o000,ambulatory,measurement 30,2018-01-01 14:17:10,NaN,NaN,135.5,93.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.753411,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,a005,return,Temporal challenge start 1,2018-01-02 14:38:34,115.0,71.0,110.0,68.0,5.0,3.0,...,0.093355,-0.938597,0.016637,-0.0025,3.002960,0.000000,0.010,0.0675,-0.813435,0.01925
381,a005,return,Temporal challenge start 2,2018-01-02 14:40:13,109.0,71.0,110.0,68.0,-1.0,3.0,...,0.007684,-3.519975,-0.005762,-0.0175,2.754045,0.000000,0.010,-0.0425,-0.500740,0.01125
382,a005,return,Temporal challenge start 3,2018-01-02 14:41:54,121.5,74.0,110.0,68.0,11.5,6.0,...,-0.069275,-4.135001,0.011637,-0.0125,4.640690,0.000000,0.010,0.0125,-0.459383,0.01025
383,a005,synthetic,Calibration average values,NaN,110.0,68.0,110.0,68.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000,0.0000,0.000000,0.00000


## Filtragem por pacientes

In [29]:
#função para particionar o df por pacientes, mostrando a medição e a anotação das pressões
def filtrarPorPaciente(df, pid):
    #args: df: dataframe com todos os dados de todos os pacientes(features.tsv)
    #pid: ID de identificação do paciente

    cols = ['pid', 'measurement', 'sbp', 'dbp']
    df_reduzido = df[df['pid'] == pid][cols]

    return df_reduzido

In [30]:
p = [f"o00{i}" for i in range(6)] 
patients_df = {pid: filtrarPorPaciente(df, pid) for pid in p} ##formando um dicionário com todas as anotações

#formando dfs para os pacientes
o000_df = patients_df['o000']

o001_df = patients_df['o001']

o002_df = patients_df['o002']

o003_df = patients_df['o003']

o004_df = patients_df['o004']

o005_df = patients_df['o005']

In [28]:
o002_df = filtrarPorPaciente(df, 'o002')
o002_df

,pid,measurement,sbp,dbp


In [31]:
meas_df = pd.concat([o000_df, o001_df, o002_df, o003_df, o004_df, o005_df])

In [32]:
meas_df

,pid,measurement,sbp,dbp
0,o000,measurement 23,NaN,NaN
1,o000,measurement 25,141.0,88.0
2,o000,measurement 27,139.0,93.0
3,o000,measurement 28,NaN,NaN
4,o000,measurement 30,NaN,NaN
...,...,...,...,...
290,o005,Sitting arm up,126.0,75.0
291,o005,Calibration average values,114.0,66.0
292,o005,Calibration closest values,115.0,65.0
293,o005,Seated calibration average values,125.0,85.0


In [40]:
df_filtrado = df[['pid', 'measurement', 'sbp', 'dbp']].query("pid.str.startswith('o00')").copy()

In [41]:
df_filtrado

,pid,measurement,sbp,dbp
0,o000,measurement 23,NaN,NaN
1,o000,measurement 25,141.0,88.0
2,o000,measurement 27,139.0,93.0
3,o000,measurement 28,NaN,NaN
4,o000,measurement 30,NaN,NaN
...,...,...,...,...
290,o005,Sitting arm up,126.0,75.0
291,o005,Calibration average values,114.0,66.0
292,o005,Calibration closest values,115.0,65.0
293,o005,Seated calibration average values,125.0,85.0


### Resultado da listagem de medições

In [33]:
##Listando as medições por paciente

p = [f'o00{i}' for i in range(6)]


medicoes_por_paciente = {pid: meas_df[meas_df['pid'] == pid]['measurement'].tolist()
                        for pid in p} #dicionário com as medições

#mostrar as medições em lista
for pid, medicoes in medicoes_por_paciente.items():
    print(f"\nPaciente {pid}:\n{medicoes}")

print("\nQuantidade de medições por paciente:")
for pid, medicoes in medicoes_por_paciente.items():
    print(f"-Paciente {pid}: {len(medicoes)}")

total = sum(len(medicoes) for medicoes in medicoes_por_paciente.values())
print(f"\nQuantidade total de medicoes: {total}")


Paciente o000:
['measurement 23', 'measurement 25', 'measurement 27', 'measurement 28', 'measurement 30', 'measurement 33', 'measurement 35', 'measurement 37', 'measurement 38', 'measurement 39', 'measurement 40', 'measurement 41', 'measurement 42', 'measurement 43', 'measurement 44', 'measurement 45', 'measurement 46', 'measurement 48', 'measurement 50', 'measurement 52', 'measurement 53', 'measurement 55', 'measurement 56', 'measurement 58', 'measurement 59', 'measurement 60', 'measurement 63', 'measurement 64', 'measurement 66', 'measurement 67', 'measurement 68', 'measurement 70', 'measurement 71', 'measurement 73', 'measurement 74', 'measurement 75', 'measurement 77', 'measurement 79', 'measurement 80', 'measurement 83', 'measurement 84', 'Cool down 1', 'Cool down 2', 'Running', 'Sitting arm down', 'Sitting arm lap', 'Sitting arm up', 'Standing arm down', 'Standing arm up', 'Supine 1', 'Supine 2', 'Walking', 'Signal check', 'Sitting arm down', 'Sitting arm lap', 'Sitting arm up',

In [43]:
quantidade = df_filtrado.groupby(['pid']).count()
quantidade

,measurement,sbp,dbp
pid,,,
o000,60,51,51
o001,58,54,54
o003,64,64,64
o004,55,51,51
o005,58,56,56


# $\text{Criação do dataframe}$

## Bibliotecas e módulos

In [54]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import numpy as np
from biosppy.signals import ecg
import os.path
import icecream as ic
import sweetviz as sv

C:\Users\User\OneDrive\Área de Trabalho\LSDi_projetos\processamento_de_sinal\VitalPy-main\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [55]:
from VitalPy_main.src.vitalpython.ppg.PPGSignal import PPGSignal
from VitalPy_main.src.vitalpython.ppg.features.FeatureExtractor import FeatureExtractor #para a extração de features

In [56]:
np.VisibleDeprecationWarning = DeprecationWarning

## Leitura de arquivos para o processamento do sinal

In [57]:
#todos os caminhos de arquivo que estão váldos após o pré processamento
fpaths = [
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_25.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_27.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_35.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_37.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_38.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_39.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_40.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_42.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_44.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_45.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_46.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_50.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_52.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_55.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_59.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_63.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_64.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_66.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_67.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_71.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_74.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_77.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_80.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_83.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_84.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.initial.Cool_down_1.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.initial.Cool_down_2.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.initial.Running.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.initial.Sitting_arm_down.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.initial.Sitting_arm_lap.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.initial.Sitting_arm_up.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.initial.Standing_arm_down.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.initial.Standing_arm_up.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.initial.Supine_1.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.initial.Supine_2.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.initial.Walking.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.return.Sitting_arm_down.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.return.Sitting_arm_lap.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.return.Sitting_arm_up.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.ambulatory.measurement_19.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.ambulatory.measurement_20.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.ambulatory.measurement_21.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.ambulatory.measurement_22.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.ambulatory.measurement_25.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.ambulatory.measurement_26.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.ambulatory.measurement_27.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.ambulatory.measurement_29.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.ambulatory.measurement_30.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.ambulatory.measurement_36.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.ambulatory.measurement_42.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.ambulatory.measurement_51.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.ambulatory.measurement_63.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.ambulatory.measurement_64.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.ambulatory.measurement_70.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.initial.Cool_down_1.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.initial.Cool_down_2.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.initial.Running.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.initial.Sitting_arm_down.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.initial.Sitting_arm_lap.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.initial.Sitting_arm_up.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.initial.Standing_arm_down.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.initial.Standing_arm_up.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.initial.Supine_1.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.initial.Supine_2.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.initial.Walking.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.return.Sitting_arm_lap.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.return.Sitting_arm_up.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o002\\o002.ambulatory.measurement_21.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o002\\o002.ambulatory.measurement_24.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o002\\o002.ambulatory.measurement_39.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o002\\o002.ambulatory.measurement_44.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_27.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_29.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_30.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_33.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_35.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_37.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_38.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_39.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_40.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_41.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_42.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_43.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_44.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_45.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_46.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_47.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_48.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_49.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_51.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_52.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_53.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_54.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_58.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_59.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_60.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.initial.Standing_arm_up.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.initial.Supine_1.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.initial.Supine_2.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.return.Sitting_arm_up.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_34.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_35.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_36.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_37.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_40.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_42.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_43.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_44.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_45.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_46.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_47.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_48.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_49.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_50.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_51.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_52.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_53.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_54.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_55.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_58.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_60.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_66.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_70.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_72.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_75.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.initial.Supine_1.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.initial.Supine_2.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.return.Signal_check.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.return.Sitting_arm_lap.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.return.Sitting_arm_up.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_34.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_37.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_38.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_43.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_44.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_46.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_48.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_49.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_50.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_51.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_52.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_53.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_54.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_55.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_56.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_57.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_59.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_60.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_61.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_62.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_69.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_71.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_74.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_76.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_77.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.initial.Cool_down_1.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.initial.Cool_down_2.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.initial.Running.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.initial.Sitting_arm_lap.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.initial.Sitting_arm_up.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.initial.Standing_arm_down.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.initial.Standing_arm_up.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.initial.Supine_1.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.initial.Supine_2.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.initial.Walking.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.return.Signal_check.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.return.Sitting_arm_lap.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.return.Sitting_arm_up.tsv",
]


In [64]:
## função feita para processar a onda
##args: o caminho do arquivo e uma lista com as features desejadas (as 38 features de interseção)
def processar_onda(path_csv, features_desejadas):
    waveform_df = pd.read_csv(path_csv, delimiter='\t') #lê a onda
    waveform_df = waveform_df.rename(columns={'optical': 'ppg', 'ekg': 'ecg'}) #renomeia as colunas
    waveform_df = waveform_df[['t', 'ppg']] #trabalha só com as colunas de tempo e ppg
    ppg = PPGSignal(signal=waveform_df, verbose=0) #faz o pré processamento
    features = ppg.extract_features() #extrai as features

    #tratamento de erro:
    if features is None:
        raise ValueError("Extração falhou: sem templates válidos") ## nao passou nos criterios do pre processamento
        
    features_filtradas = {
        k: features.get(k, None)
        for k in features_desejadas if k in features}
    return features_filtradas

In [65]:
# Vetor com o caminho dos sinais
#filepath = tsv_paths
filepath = []

# Vetor de features a serem analisadas
features_selecionadas = [
 'temp_key_tr_md_s',
 'temp_key_t_a_md',
 'temp_key_t_a_b',
 'temp_key_t_a_d',
  'temp_key_slp_md',
 'temp_key_id2_o',
 'temp_key_id2_b',
 'temp_key_id2_s',
 'temp_key_id2_ip',
 'temp_key_id2_d',
 'temp_key_a_a_s',
 'temp_key_a_md_s',
 'temp_max_p',
 'temp_zc1d',
 'bw_dbw10',
 'bw_bw33',
 'bw_bwr50',
 'bw_dbw90',
 'poly_coeff_coef4',
 'spect_spectral_skewness',
 'f1__',
 'spectral_distance__',
 'median_frequency__',
 'spectral_maxpeaks__',
 'spectral_roll_off__',
 'human_range_energy__',
 'power_bandwidth__',
 'wavelet_entropy__'
]



# $\text{Extração de features por paciente}$

## $\text{Extração das features para o paciente o000}$

In [46]:
#lista com todos os caminhos do paciente o000 que passaram no pre processamento 
o00path = [
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_25.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_27.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_35.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_37.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_38.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_39.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_40.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_42.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_44.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_45.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_46.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_50.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_52.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_55.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_59.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_63.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_64.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_66.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_67.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_71.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_74.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_77.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_80.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_83.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_84.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.initial.Cool_down_1.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.initial.Cool_down_2.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.initial.Running.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.initial.Sitting_arm_down.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.initial.Sitting_arm_lap.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.initial.Sitting_arm_up.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.initial.Standing_arm_down.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.initial.Standing_arm_up.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.initial.Supine_1.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.initial.Supine_2.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.initial.Walking.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.return.Sitting_arm_down.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.return.Sitting_arm_lap.tsv",
r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.return.Sitting_arm_up.tsv",
]

In [47]:
filepath = o00path #o vetor dos caminhos (geral) vira só os do paciente o000
data = [] #faz uma lista com os arquivos

for i, path in enumerate(o00path):
    try:
        # Passa o caminho do arquivo atual para a função processar_onda
        linha = processar_onda(path, features_selecionadas)
        data.append(linha)
    except Exception as e:
        print(f"Erro na onda {i+1} ({path}): {e}")


Erro na onda 1 (aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_25.tsv): name 'PPGSignal' is not defined
Erro na onda 2 (aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_27.tsv): name 'PPGSignal' is not defined
Erro na onda 3 (aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_35.tsv): name 'PPGSignal' is not defined
Erro na onda 4 (aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_37.tsv): name 'PPGSignal' is not defined
Erro na onda 5 (aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_38.tsv): name 'PPGSignal' is not defined
Erro na onda 6 (aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.ambulatory.measurement_39.tsv): name 'PPGSignal' is not defined
Erro na onda 7 (aurorabp-sample-data-main/sample/measurements_oscillometric\\o000\\o000.

In [19]:
o000_features = pd.DataFrame(data)
o000_features.index = [f"onda_{i+1}" for i in range(len(o000_features))]
o000_features['pid'] = 'o000'
cols = ['pid'] + [col for col in o000_features.columns if col != 'pid']
o000_features = o000_features[cols]
o000_features

,pid,temp_key_tr_md_s,temp_key_t_a_md,temp_key_t_a_b,temp_key_t_a_d,temp_key_slp_md,temp_key_id2_o,temp_key_id2_b,temp_key_id2_s,temp_key_id2_ip,...,poly_coeff_coef4,spect_spectral_skewness,f1__,spectral_distance__,median_frequency__,spectral_maxpeaks__,spectral_roll_off__,human_range_energy__,power_bandwidth__,wavelet_entropy__
onda_1,o000,0.495073,-0.052250,-0.092667,-0.556667,18.780075,305.150761,-479.190654,-536.827714,244.586519,...,1.363804e+10,25.229503,0.034332,-1.333397e+09,1.757593,49,9.420697,0.749769,1.757099,1.745904
onda_2,o000,0.468891,-0.064375,-0.115000,-0.551000,16.693497,349.228281,-369.097965,-406.308738,265.024361,...,2.522347e+09,20.090578,0.118256,-1.064728e+09,1.732352,306,157.968818,0.616718,3.354978,1.769589
onda_3,o000,0.458132,-0.057565,-0.100957,-0.545826,18.586578,412.785331,-452.507003,-370.639063,358.559878,...,6.770105e+09,24.841284,0.038147,-8.209230e+08,1.828154,1272,104.433272,0.819959,3.045995,1.760694
onda_4,o000,0.482112,-0.046632,-0.091263,-0.623158,16.787067,197.798211,-466.724554,-286.346345,134.488040,...,-2.876622e+10,24.282042,0.076294,-3.894706e+09,2.989301,496,138.687854,0.751749,1.651463,1.746047
onda_5,o000,0.485765,-0.045667,-0.086000,-0.787000,11.393372,106.937914,-305.245435,-192.879010,30.882947,...,-4.007070e+08,29.145436,0.221252,-4.390716e+08,2.305919,419,153.535742,0.946720,1.344086,1.815248
onda_6,o000,0.461961,-0.045667,-0.088667,-0.746333,13.815340,126.561524,-395.196749,-138.917340,106.267891,...,-4.151545e+09,29.425544,0.099182,-2.656184e+08,1.865672,216,114.013267,0.867948,1.864126,1.787570
onda_7,o000,0.451902,-0.049143,-0.094286,-0.864286,12.092696,106.910492,-299.625030,-125.410906,54.524516,...,-3.445770e+08,30.177691,0.267029,-7.449764e+08,2.007412,492,144.688079,0.871516,1.542734,1.786496
onda_8,o000,0.448647,-0.049091,-0.104545,-0.676909,14.937813,213.764284,-332.550852,-286.061324,141.283979,...,-2.534555e+09,26.862806,0.297546,-7.780375e+08,1.506781,801,76.594676,0.817325,2.510040,1.761168
onda_9,o000,0.430694,-0.059579,-0.109789,-0.574737,17.626970,369.106048,-374.996913,-347.164167,302.761390,...,-7.829322e+10,22.931669,0.045776,-1.311250e+09,3.077735,168,153.710869,0.803264,2.812445,1.759355
onda_10,o000,0.551332,-0.037800,-0.057200,-0.529000,23.078293,384.678760,-716.852975,-791.069089,332.173073,...,-8.279263e+10,20.269411,0.373840,-1.274484e+08,2.789400,567,65.027894,0.579103,3.484321,1.765564


## $\text{Extração das features para o paciente o001}$

In [60]:
o01path = [
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.ambulatory.measurement_19.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.ambulatory.measurement_20.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.ambulatory.measurement_21.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.ambulatory.measurement_22.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.ambulatory.measurement_25.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.ambulatory.measurement_26.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.ambulatory.measurement_27.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.ambulatory.measurement_29.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.ambulatory.measurement_30.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.ambulatory.measurement_36.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.ambulatory.measurement_42.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.ambulatory.measurement_51.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.ambulatory.measurement_63.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.ambulatory.measurement_64.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.ambulatory.measurement_70.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.initial.Cool_down_1.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.initial.Cool_down_2.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.initial.Running.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.initial.Sitting_arm_down.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.initial.Sitting_arm_lap.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.initial.Sitting_arm_up.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.initial.Standing_arm_down.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.initial.Standing_arm_up.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.initial.Supine_1.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.initial.Supine_2.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.initial.Walking.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.return.Sitting_arm_lap.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o001\\o001.return.Sitting_arm_up.tsv"
]

In [66]:
filepath =o01path
data = []

for i, path in enumerate(o01path):
    try:
        # Passa o caminho do arquivo atual para a função processar_onda
        linha = processar_onda(path, features_selecionadas)
        data.append(linha)
    except Exception as e:
        print(f"Erro na onda {i+1} ({path}): {e}")

o001_features = pd.DataFrame(data)
o001_features.index = [f"onda_{i+1}" for i in range(len(o001_features))]
o001_features['pid'] = 'o001'
cols = ['pid'] + [col for col in o001_features.columns if col != 'pid']
o001_features = o001_features[cols]
o001_features

,pid,temp_key_tr_md_s,temp_key_t_a_md,temp_key_t_a_b,temp_key_t_a_d,temp_key_slp_md,temp_key_id2_o,temp_key_id2_b,temp_key_id2_s,temp_key_id2_ip,...,poly_coeff_coef4,spect_spectral_skewness,f1__,spectral_distance__,median_frequency__,spectral_maxpeaks__,spectral_roll_off__,human_range_energy__,power_bandwidth__,wavelet_entropy__
onda_1,o001,0.439147,-0.049333,-0.092667,-0.806667,13.077460,116.657728,-333.234440,-99.610450,115.769782,...,-1.783359e+09,32.840997,0.099182,-4.146062e+08,1.160991,576,109.133127,0.892716,1.546193,1.773744
onda_2,o001,0.411870,-0.049667,-0.092833,-0.809333,14.702356,216.422047,-343.382501,-135.230758,141.492212,...,1.183575e+09,30.448154,0.057220,-1.607050e+09,2.251370,653,132.439311,0.928736,1.369863,1.770064
onda_3,o001,0.422959,-0.048000,-0.092000,-0.851000,12.919203,93.966174,-319.811593,-93.216175,41.152774,...,-1.901253e+09,36.568799,0.198364,-6.520098e+08,1.478197,400,76.496674,0.918514,1.845700,1.793784
onda_4,o001,0.438068,-0.046933,-0.090667,-0.833733,13.529997,109.973466,-362.042999,-96.820510,74.705680,...,-3.254975e+08,33.661646,0.034332,-5.537570e+09,1.204456,44,45.242397,0.926329,1.354402,1.747493
onda_5,o001,0.463234,-0.056000,-0.106667,-0.991778,9.134207,34.287731,-182.603032,-145.780730,27.997802,...,1.045535e+08,30.153732,0.125885,-3.098111e+09,1.449275,901,116.666667,0.915338,1.241465,1.781693
onda_6,o001,0.379691,-0.063333,-0.119333,-0.986333,10.205307,91.724291,-162.044867,-136.513117,38.018450,...,1.868473e+08,39.433230,0.091553,-4.242586e+08,0.970874,266,69.255663,0.967833,1.131587,1.782782
onda_7,o001,0.513481,-0.048333,-0.087667,-0.964333,6.554831,21.650868,-185.659760,-86.991012,23.160630,...,-1.993426e+08,31.314100,0.122070,-1.338032e+09,1.057402,630,55.891239,0.945284,1.056444,1.790079
onda_8,o001,0.467792,-0.058000,-0.124000,-0.891000,10.050410,59.253587,-221.171353,-185.575437,34.164210,...,-3.038096e+08,42.217497,0.076294,-2.692485e+08,1.207729,271,55.210490,0.964961,1.206481,1.779446
onda_9,o001,0.463108,-0.064667,-0.135333,-0.934000,10.374592,94.953593,-211.864466,-121.670652,39.790662,...,-2.225228e+07,31.197550,0.076294,-5.806650e+08,1.002674,296,5.848930,0.924077,1.335559,1.769453
onda_10,o001,0.390371,-0.052500,-0.097750,-0.721750,13.988173,222.312301,-269.905343,-178.185843,136.280342,...,4.529220e+09,26.919200,0.114441,-4.105109e+08,1.318392,551,75.972314,0.870488,1.646904,1.791000


## $\text{Extração das features para o paciente o002}$

In [22]:
o02path = [
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o002\\o002.ambulatory.measurement_21.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o002\\o002.ambulatory.measurement_24.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o002\\o002.ambulatory.measurement_39.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o002\\o002.ambulatory.measurement_44.tsv"
]

In [23]:
filepath = o02path
data = []

for i, path in enumerate(o02path):
    try:
        # Passa o caminho do arquivo atual para a função processar_onda
        linha = processar_onda(path, features_selecionadas)
        data.append(linha)
    except Exception as e:
        print(f"Erro na onda {i+1} ({path}): {e}")

o002_features = pd.DataFrame(data)
o002_features.index = [f"onda_{i+1}" for i in range(len(o002_features))]
o002_features['pid'] = 'o002'
cols = ['pid'] + [col for col in o002_features.columns if col != 'pid']
o002_features = o002_features[cols]
o002_features

,pid,temp_key_tr_md_s,temp_key_t_a_md,temp_key_t_a_b,temp_key_t_a_d,temp_key_slp_md,temp_key_id2_o,temp_key_id2_b,temp_key_id2_s,temp_key_id2_ip,...,poly_coeff_coef4,spect_spectral_skewness,f1__,spectral_distance__,median_frequency__,spectral_maxpeaks__,spectral_roll_off__,human_range_energy__,power_bandwidth__,wavelet_entropy__
onda_1,o002,0.318888,-0.058250,-0.160000,-0.893500,8.987283,159.088978,-211.818860,-141.773717,99.144215,...,-1.103882e+09,30.767319,0.068665,-3.808305e+08,1.321353,709,120.639535,0.969967,1.056245,1.790198
onda_2,o002,0.348457,-0.063000,-0.142000,-0.867750,8.204042,169.598103,-168.845319,-162.223074,141.479047,...,-1.029909e+09,30.721035,0.061035,-2.677745e+08,1.081666,234,99.918875,0.992729,0.270197,1.797224
onda_3,o002,0.340396,-0.047000,-0.098333,-0.744333,12.211536,270.375101,-323.742592,-171.751111,147.987051,...,1.231725e+09,23.904355,0.236511,-1.117829e+08,1.262626,565,44.191919,0.892924,1.261034,1.790297
onda_4,o002,0.340277,-0.049143,-0.097143,-0.872857,11.106542,163.206632,-280.840928,-166.235915,81.870100,...,-7.760343e+08,32.998765,0.076294,-2.558650e+08,1.689189,596,101.812039,0.957433,1.227747,1.801258


## $\text{Extração das features para o paciente o003}$

In [24]:
o03path = [
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_27.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_29.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_30.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_33.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_35.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_37.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_38.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_39.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_40.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_41.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_42.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_43.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_44.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_45.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_46.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_47.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_48.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_49.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_51.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_52.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_53.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_54.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_58.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_59.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.ambulatory.measurement_60.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.initial.Standing_arm_up.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.initial.Supine_1.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.initial.Supine_2.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o003\\o003.return.Sitting_arm_up.tsv"
]

In [25]:

data = []

for i, path in enumerate(o03path):
    try:
        # Passa o caminho do arquivo atual para a função processar_onda
        linha = processar_onda(path, features_selecionadas)
        data.append(linha)
    except Exception as e:
        print(f"Erro na onda {i+1} ({path}): {e}")

o003_features = pd.DataFrame(data)
o003_features.index = [f"onda_{i+1}" for i in range(len(o003_features))]
o003_features['pid'] = 'o003'
cols = ['pid'] + [col for col in o003_features.columns if col != 'pid']
o003_features = o003_features[cols]
o003_features

,pid,temp_key_tr_md_s,temp_key_t_a_md,temp_key_t_a_b,temp_key_t_a_d,temp_key_slp_md,temp_key_id2_o,temp_key_id2_b,temp_key_id2_s,temp_key_id2_ip,...,poly_coeff_coef4,spect_spectral_skewness,f1__,spectral_distance__,median_frequency__,spectral_maxpeaks__,spectral_roll_off__,human_range_energy__,power_bandwidth__,wavelet_entropy__
onda_1,o003,0.547740,-0.039400,-0.061600,-0.581400,21.642716,308.455720,-723.872955,-623.431299,283.623322,...,-1.506641e+11,16.573077,0.198364,-2.891096e+08,3.192848,631,151.819923,0.459565,3.668262,1.784741
onda_2,o003,0.545423,-0.041000,-0.074500,-0.608167,17.173741,161.172115,-634.179706,-391.976842,117.587219,...,-2.199534e+10,24.303322,0.152588,-7.869387e+08,2.503756,425,129.193791,0.601948,2.752753,1.789538
onda_3,o003,0.480122,-0.043538,-0.083692,-0.672308,13.961027,132.083954,-441.384740,-161.175356,103.880753,...,-1.077986e+10,29.181149,0.247955,-1.920963e+09,1.567071,838,57.563728,0.822944,1.670844,1.765435
onda_4,o003,0.369197,-0.058667,-0.114133,-0.663200,14.659265,251.216162,-263.751417,-193.036828,204.316059,...,-2.008084e+10,29.567315,0.045776,-2.311481e+09,2.505783,37,138.010794,0.877623,1.541129,1.753761
onda_5,o003,0.547343,-0.046000,-0.078000,-0.635333,16.329688,342.976822,-635.593072,-317.290780,188.274633,...,3.274549e+09,17.050057,0.549316,-1.519914e+08,2.892960,349,159.112825,0.558711,2.888782,1.835612
onda_6,o003,0.480083,-0.040800,-0.079000,-0.615800,15.604715,151.254327,-544.706299,-120.907533,42.435748,...,-3.551185e+10,23.935736,0.244141,-1.075053e+09,1.788909,542,88.401908,0.765668,2.234803,1.779449
onda_7,o003,0.504124,-0.045231,-0.087077,-0.675077,13.779798,124.259052,-435.033700,-192.021729,83.266256,...,-1.353631e+10,31.572353,0.308990,-1.933590e+09,2.801993,413,149.335824,0.811392,1.451981,1.765211
onda_8,o003,0.430624,-0.045273,-0.087455,-0.752182,14.273751,160.054729,-365.354774,-150.141046,81.949640,...,-3.875286e+09,23.818979,0.038147,-1.966839e+09,2.385279,773,135.620173,0.877217,1.475931,1.778395
onda_9,o003,0.380893,-0.060267,-0.138133,-0.785467,11.126239,145.596771,-186.176588,-152.275703,102.477516,...,-3.357076e+08,35.451205,0.034332,-2.594001e+09,1.201923,867,9.535256,0.953615,1.281640,1.759726
onda_10,o003,0.350885,-0.055875,-0.167000,-0.815875,10.572801,110.168933,-169.726410,-125.369032,65.003424,...,-3.460810e+08,36.334728,0.049591,-4.625066e+09,1.163806,45,107.215595,0.962919,1.236004,1.744515


## $\text{Extração das features para o paciente o004}$

In [26]:
o04path = [
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_34.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_35.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_36.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_37.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_40.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_42.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_43.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_44.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_45.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_46.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_47.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_48.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_49.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_50.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_51.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_52.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_53.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_54.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_55.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_58.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_60.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_66.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_70.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_72.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.ambulatory.measurement_75.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.initial.Supine_1.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.initial.Supine_2.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.return.Signal_check.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.return.Sitting_arm_lap.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o004\\o004.return.Sitting_arm_up.tsv"
]

In [27]:

data = []

for i, path in enumerate(o04path):
    try:
        # Passa o caminho do arquivo atual para a função processar_onda
        linha = processar_onda(path, features_selecionadas)
        data.append(linha)
    except Exception as e:
        print(f"Erro na onda {i+1} ({path}): {e}")

o004_features = pd.DataFrame(data)
o004_features.index = [f"onda_{i+1}" for i in range(len(o004_features))]
o004_features['pid'] = 'o004'
cols = ['pid'] + [col for col in o004_features.columns if col != 'pid']
o004_features = o004_features[cols]
o004_features

,pid,temp_key_tr_md_s,temp_key_t_a_md,temp_key_t_a_b,temp_key_t_a_d,temp_key_slp_md,temp_key_id2_o,temp_key_id2_b,temp_key_id2_s,temp_key_id2_ip,...,poly_coeff_coef4,spect_spectral_skewness,f1__,spectral_distance__,median_frequency__,spectral_maxpeaks__,spectral_roll_off__,human_range_energy__,power_bandwidth__,wavelet_entropy__
onda_1,o004,0.461671,-0.055368,-0.101053,-0.604421,16.727057,251.895418,-407.194274,-360.553366,192.062144,...,2.594142e+10,23.225963,0.049591,-1.292571e+09,1.726406,956,14.633344,0.813238,1.643655,1.753753
onda_2,o004,0.579964,-0.046333,-0.082667,-0.578000,13.151498,136.410502,-483.977419,-325.920057,146.482074,...,-2.064739e+08,26.564193,0.053406,-5.767377e+08,1.525941,343,75.025432,0.769423,2.795426,1.761680
onda_3,o004,0.467314,-0.054000,-0.105250,-0.637875,14.061020,164.016721,-330.386197,-269.667179,118.564153,...,-1.257780e+10,31.747616,0.049591,-1.669845e+09,1.640540,352,117.298578,0.803321,1.548834,1.756672
onda_4,o004,0.441190,-0.049263,-0.092421,-0.588000,16.964899,306.949130,-407.061350,-286.829713,241.986849,...,-5.134874e+10,25.288332,0.106812,-1.693342e+09,1.679543,1085,111.269735,0.811666,2.770316,1.754727
onda_5,o004,0.452082,-0.046000,-0.088533,-0.644933,16.657050,212.704488,-456.909718,-198.206612,160.871105,...,-3.081664e+10,31.198390,0.041962,-1.762459e+09,1.740139,369,54.331013,0.826007,1.545894,1.753117
onda_6,o004,0.449872,-0.046632,-0.088421,-0.643263,15.574998,174.168652,-388.887316,-225.946476,136.272801,...,-3.010873e+10,35.224083,0.034332,-2.421804e+09,1.522997,708,9.747182,0.839842,1.598417,1.746288
onda_7,o004,0.449083,-0.047429,-0.090857,-0.601905,16.520851,228.586292,-381.181520,-308.479775,189.452122,...,-3.674621e+10,27.944386,0.038147,-5.541982e+09,1.773050,44,91.238180,0.816170,1.698671,1.742713
onda_8,o004,0.443063,-0.053200,-0.133800,-0.665300,12.263009,88.852840,-249.577314,-273.071799,53.497651,...,-3.830487e+08,36.812073,0.030518,-5.496854e+09,1.467710,43,86.175566,0.879387,1.537172,1.738818
onda_9,o004,0.438249,-0.050857,-0.112571,-0.701857,12.121509,90.980380,-235.347551,-239.560220,66.747902,...,-4.834976e+09,35.596562,0.114441,-3.111169e+09,1.507727,1275,100.735017,0.820844,2.543331,1.759329
onda_10,o004,0.483597,-0.056750,-0.131750,-0.676625,11.485775,71.469861,-243.259238,-222.200382,50.425784,...,-1.717884e+09,38.234807,0.041962,-2.826522e+09,1.353180,38,54.296346,0.899293,1.437268,1.744955


## $\text{Extração das features para o paciente o005}$

In [28]:
o05path = [
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_34.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_37.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_38.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_43.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_44.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_46.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_48.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_49.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_50.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_51.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_52.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_53.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_54.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_55.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_56.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_57.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_59.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_60.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_61.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_62.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_69.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_71.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_74.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_76.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.ambulatory.measurement_77.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.initial.Cool_down_1.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.initial.Cool_down_2.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.initial.Running.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.initial.Sitting_arm_lap.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.initial.Sitting_arm_up.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.initial.Standing_arm_down.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.initial.Standing_arm_up.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.initial.Supine_1.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.initial.Supine_2.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.initial.Walking.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.return.Signal_check.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.return.Sitting_arm_lap.tsv",
    r"aurorabp-sample-data-main/sample/measurements_oscillometric\\o005\\o005.return.Sitting_arm_up.tsv",
]

In [29]:

data = []

for i, path in enumerate(o05path):
    try:
        # Passa o caminho do arquivo atual para a função processar_onda
        linha = processar_onda(path, features_selecionadas)
        data.append(linha)
    except Exception as e:
        print(f"Erro na onda {i+1} ({path}): {e}")

o005_features = pd.DataFrame(data)
o005_features.index = [f"onda_{i+1}" for i in range(len(o005_features))]
o005_features['pid'] = 'o005'
cols = ['pid'] + [col for col in o005_features.columns if col != 'pid']
o005_features = o005_features[cols]
o005_features

,pid,temp_key_tr_md_s,temp_key_t_a_md,temp_key_t_a_b,temp_key_t_a_d,temp_key_slp_md,temp_key_id2_o,temp_key_id2_b,temp_key_id2_s,temp_key_id2_ip,...,poly_coeff_coef4,spect_spectral_skewness,f1__,spectral_distance__,median_frequency__,spectral_maxpeaks__,spectral_roll_off__,human_range_energy__,power_bandwidth__,wavelet_entropy__
onda_1,o005,0.505090,-0.043000,-0.076200,-0.647600,20.852175,281.219571,-676.631736,-427.624837,202.661355,...,-5.135733e+10,25.618940,0.076294,-4.842142e+08,2.912056,689,46.884100,0.665992,1.745708,1.753815
onda_2,o005,0.456322,-0.055000,-0.110500,-0.778125,14.569743,144.496121,-331.798499,-188.560654,107.138136,...,-2.542773e+09,32.255706,0.034332,-3.764216e+09,1.441141,1568,18.203883,0.889501,1.592115,1.751285
onda_3,o005,0.462851,-0.051429,-0.101286,-0.778571,14.606483,138.227654,-359.578043,-177.770725,85.145999,...,-4.301493e+09,35.335769,0.019073,-3.386710e+09,2.071823,615,75.707873,0.904260,1.380739,1.751347
onda_4,o005,0.499340,-0.049667,-0.090000,-0.745000,16.477260,223.548695,-500.248974,-260.444811,141.132703,...,-6.998991e+09,25.107539,0.045776,-6.474831e+08,2.516779,730,116.191275,0.787580,2.515723,1.778237
onda_5,o005,0.349232,-0.064000,-0.119333,-0.755333,14.550666,282.826040,-276.227878,-138.794372,202.948252,...,1.593861e+09,27.485177,0.122070,-1.415657e+08,2.570694,393,161.096829,0.858647,1.497647,1.824723
onda_6,o005,0.395076,-0.046933,-0.086800,-0.755733,13.302413,132.099151,-296.287074,-140.406104,89.905428,...,-2.076453e+09,32.839527,0.049591,-3.241211e+09,1.409619,1171,47.097844,0.922050,1.492042,1.757098
onda_7,o005,0.450764,-0.047500,-0.092125,-0.726125,16.234945,181.326827,-412.409628,-206.446163,121.445672,...,-6.106857e+09,34.622199,0.038147,-4.247386e+09,1.706207,439,6.499838,0.829862,1.461751,1.743566
onda_8,o005,0.512751,-0.057500,-0.093000,-0.616250,19.953902,351.608518,-567.842119,-495.893296,270.947738,...,-4.040603e+10,18.131718,0.236511,-1.386944e+08,1.737452,223,80.308880,0.666998,2.699576,1.766693
onda_9,o005,0.496404,-0.042353,-0.083294,-0.741882,16.077535,119.693453,-521.364672,-234.916703,60.766977,...,-6.763605e+09,31.032376,0.015259,-5.572942e+09,2.222881,789,18.005335,0.820034,1.481262,1.743609
onda_10,o005,0.462561,-0.045750,-0.089375,-0.772250,15.429954,158.878075,-394.730321,-267.678729,107.684270,...,-3.911235e+08,29.191850,0.034332,-3.913155e+09,2.208346,1229,16.600670,0.916112,1.370280,1.749472


# $\text{Junção dos dataframes}$

In [61]:
dataframes = [o001_features, o002_features, o003_features, o004_features, o005_features]
patient_features = pd.concat(dataframes)


In [64]:
patient_features.to_csv('p_features.csv')